# Simulateur numérique de l’équation de Schrödinger périodique

## I - Introduction
Nous nous intéressons ici à la résolution de l'équation de Schrödinger 1D et 2D dans un domaine:
$$ \Omega = \prod_{i=1}^{d}\left]-\frac{L_i}{2}, \frac{L_i}{2}\right[\subset \mathbb{R}^d,\quad d\in \{1, 2\}$$ 
avec des conditions périodiques définies par :
\begin{equation*}
H_\#^2(\Omega) := \left\{ \psi \in H_{loc}^2(\Omega), ~ \forall i \in \{1, d\}, ~ \psi\vert_{x_i = -\frac{L_i}{2}} = \psi\vert_{x_i = \frac{L_i}{2}} \right\}.
\end{equation*}

L'équation de Schrödinger périodique s'écrit, pour un potentiel $V$ suffisamment régulier :
$$\left|\begin{aligned}
&\text{Trouver } \psi \in C^0\left(\left[0, T\right], H_\#^2(\Omega)\right) \text{ telle que :} \\
&\quad i\partial_t \psi(\boldsymbol{x}, t) = -\frac{1}{2}\Delta \psi(\boldsymbol{x}, t) + V(\boldsymbol{x}, t)\psi(\boldsymbol{x}, t)&, &\text{ dans } \Omega \times [0, T] \\
&\quad \psi(\cdot, 0) = \psi_0&, &\text{ dans } \Omega
\end{aligned}\right.$$


Pour la résolution numérique, nous mettrons en œuvre une méthode pseudo-spectrale pour la discrétisation spatiale, couplée à une méthode de splitting pour la discrétisation temporelle.

## II - Discrétisation spatiale
Considérons l'espace fonctionnel des fonctions $L^2$ périodiques sur $\Omega$ :
$$
    L_\#^2(\Omega) = \left\{u \in L^2(\Omega),~ u(x + L) = u(x),~ \text{p.p. } x \in \Omega \right\}
$$
muni du produit scalaire :
$$
    \left<u|v\right> = \int_\Omega \overline{u}(x)v(x) ~ dx.
$$

Nous introduisons les modes de Fourier, qui constituent une base orthonormée de $L_\#^2(\Omega)$, définis par :
$$
    \begin{cases}
        \forall n \in \Z, ~ e_n(x) := \left<x|e_n\right> = \frac{1}{\sqrt{L}}e^{ik_nx}, \quad k_n = \frac{2n\pi}{L}, \\
        \forall n, m \in \Z, ~\left<e_m|e_n\right> = \delta_{m,n}.
    \end{cases}
$$

Etant donné que la solution $\psi(\boldsymbol{x}, t)$ est $L$-périodique, nous la décomposons sur cette base comme suit :
$$
    \Ket{\psi(t)} = \sum_{n \in \Z}c_n(t)\Ket{e_n},
$$
où les coefficients de Fourier sont :
$$
    c_n(t) = \left<e_n|\psi(t)\right> = \frac{1}{\sqrt{L}}\int_\Omega \psi(x,t) e^{-ik_nx}~d x = \sqrt{\frac{2\pi}{L}}\mathcal{F}(\psi)(k_n, t).
$$

Afin de discrétiser le problème, nous introduisons l’espace des premiers modes de Fourier :
$$
    X_{N_x} = \text{Vect}\left(\Ket{e_n}, |n| \leq N_x\right),
$$
et nous projetons la solution \(\Ket{\psi(t)}\) sur cet espace au moyen du projecteur \(\hat{P}_{N_x}\) :
$$
    \hat{P}_{N_x} = \sum_{|i| \leq N_x} \Ket{e_i}\Bra{e_i}.
$$

La méthode spectrale consiste alors à approcher $\Ket{\psi(t)}$ par une solution discrète $\Ket{\psi_{N_x}(t)} \in X_{N_x}$, obtenue en résolvant le problème projeté :
$$
    \begin{cases}
        i\frac{d}{d t}\Ket{\psi_{N_x}(t)} = \hat{P}_{N_x}\hat{H}\hat{P}_{N_x}\Ket{\psi_{N_x}(t)}, \\
        \Ket{\psi_{N_x}(0)} = \hat{P}_{N_x}\Ket{\psi_0}.
    \end{cases}
$$

L’opérateur projeté $\hat{P}_{N_x}\hat{H}\hat{P}_{N_x}$ s’écrit comme suit :
\begin{align*}
    \hat{P}_{N_x}\hat{H}\hat{P}_{N_x} &= \sum_{|i| \leq N_x} \Ket{e_i}\Bra{e_i} \left(\frac{\hat{p}^2}{2} + \hat{V}(t)\right)\sum_{|j| \leq N_x} \Ket{e_j}\Bra{e_j}, \\
    &= \sum_{|i| \leq N_x}\sum_{|j| \leq N_x}\left(\frac{k_j^2}{2}\delta_{i,j} + v_{j-i}(t)\right)\Ket{e_i}\Bra{e_j},
\end{align*}
où $v_{j-i}(t) := \Bra{e_i}\hat{V}(t)\Ket{e_j} = \sqrt{\frac{2\pi}{L}}\mathcal{F}(V)(k_j -k_i, t)$.

Enfin, les coefficients \(c_n(t)\) peuvent être approchés numériquement par la transformée de Fourier discrète, donnée par :
$$
    c_n(t) \approx \frac{\sqrt{L}}{N_x}\sum_{i = 0}^{N_x - 1} \psi(x_i, t)e^{-ik_nx_i} = \frac{\sqrt{L}}{N_x}\Psi(k_n, t) = \tilde{c}_n(t),
$$
où $\Psi(k_n, t)$ est la transformée de Fourier discrète de $\Ket{\psi_{N_x}(t)}$.

On a pu vérifier que cette méthode permettait bien de calculer l'action de $\Delta$ sur des fonctions et que cette méthode de calcul est plus rapide que la fonction `scipy.linalg.dft`. Nous ne présentons pas ici tout cela car le coeur de notre projet a été l'extension en 2D de notre solveur.

## III - Discrétisation temporelle
On utilise ici une méthode dite de splitting. On subidivide $[0,T]$ en $N_t$ points et à chaque itération :
- on propage le terme d'énergie potentielle en appliquant $e^{-i\mathbb{V}(t)\Delta t}$ $\left(\mathbb{V}_{ij}(t) = v_{j-i}(t)\right)$
- on passe dans le domaine des impulsions en appliquant la FFT
- on propage le terme d'énergie cinétique en appliquant $e^{-i \boldsymbol{k}^2\Delta t}$ aux coefficient de Fourier
- on revient au domaine des positions en appliquant la iFFT

## IV - Choix des paramètres
On se propose ici de simuler :
- une particule libre
- une particule dans un potentiel harmonique
- une particule face à une barrière de potentiel

Dans tous ces cas, il nous faut choisir des paramètres cohérents pour s'assurer de mettre en évidence les phénomènes que l'on souhaite étudier.

On va résonner de manière constructive. On choisira tout le temps un paquet d'onde gaussien comme condition initiale [1]:
$$ 
    \psi(\boldsymbol{x}) = \frac{1}{C} e^{i \boldsymbol{k}_0 \cdot \boldsymbol{x}}e^{-\frac{(\boldsymbol{x} - \boldsymbol{x}_0)^2}{4\sigma^2}}
$$
on a normalisé par $C = \|\psi\|_{L^2(\Omega)}$ du fait qu'il n'y ait pas de relation explicite dans le cas où $\Omega \neq \mathbb{R}^d$.

On a, pour ce paquet d'onde gaussien (la saturation de la relation de Heisenberg) et :
- la relation de dispersion : $$ \omega = \frac{\hbar \boldsymbol{k}^2}{2m} $$
- la relation de de Broglie : $$ \boldsymbol{p} = \hbar \boldsymbol{k},~ \lambda = \frac{2\pi}{|\boldsymbol{k}|}$$
- la vitesse de phase : $$ v_\phi(\boldsymbol{k}) = \frac{\omega}{|\boldsymbol{k}|}$$
- la vitesse de groupe : $$ v_g(\boldsymbol{k}_0) = 2v_\phi(\boldsymbol{k}_0)$$
- la relation de Heisenberg temps-énergie : $$ \tau \Delta E \geq \frac{\hbar}{2} $$

On choisit les paramètres de notre simulation de la manière suivante (en 1D pour simplifier mais le cas 2D est identique):
- $ dx << \lambda << \sigma << L $
- $ \frac{2 \pi}{\omega} << \frac{L}{(\boldsymbol{k}_0)} $

## V - Résultats sur des cas tests
<video controls src = "results/wall_2D_full_reflexion_probability_density.mp4" width = "500">
cizdci


<video controls src = "results/wall_2D_full_reflexion_real_part.mp4" width = "500">

## Références
[1] Cohen-Tannoudji, C., Diu, B., Laloe, F. (2018). Mécanique Quantique - Tome 1: Nouvelle édition. Allemagne: EDP sciences.